<a href="https://colab.research.google.com/github/2311556517-jpg/cuong/blob/main/chatbot4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets peft accelerate trl evaluate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install fsspec[http]==2025.3.0 datasets==4.0.0 gcsfs==2025.3.0

In [ ]:
# !pip install -q \
#   transformers==4.40.1 \
#   datasets==2.19.1 \
#   peft==0.10.0 \
#   accelerate==0.29.3 \
#   trl==0.8.6 \
#   evaluate==0.4.1 \
#   bitsandbytes==0.42.0 \
#   fsspec==2023.6.0 \
#   gcsfs==2023.6.0

In [3]:
# Import thư viện cần thiết
import os
import torch
os.environ["WANDB_DISABLED"] = "true"

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer
#from trl.utils import get_peft_config  # Tùy chọn nếu cần dùng


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#chuẩn bị datasets
#https://huggingface.co/bkai-foundation-models/vietnamese-llama2-7b-120GB(không bó buộc thư viện nội bộ)
# template của prompt là
#prompt =
#[
#  {
#   "task": "mô tả",
#   "input": "Hãy mô tả vẻ đẹp của ruộng bậc thang ở Tây Bắc.",
#    "output": "Ruộng bậc thang Tây Bắc như những dải lụa vàng óng ả, xếp lớp nối tiếp nhau trên triền núi, tạo nên khung cảnh vừa kỳ vĩ vừa thơ mộng."
#  },
#  {
#    "task": "trả lời câu hỏi",
#    "input": "Ruộng bậc thang nổi tiếng nhất Việt Nam ở đâu?",
#    "output": "Ruộng bậc thang nổi tiếng nhất Việt Nam nằm ở tỉnh Yên Bái, đặc biệt là vùng Mù Cang Chải."
#  }
#]

#---để convert data về đúng với format--


In [4]:
#convert data gốc về format của bkai-foundation-models/vietnamese-llama2-7b-40GB.
from datasets import load_dataset, DatasetDict

# Load local JSON file
transformed_dataset = load_dataset('json', data_files='/all_data.json')

# Split the dataset into training and testing sets
train_test_split = transformed_dataset['train'].train_test_split(test_size=0.1)
transformed_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


# Xem trước dữ liệu
print(transformed_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['text'],
        num_rows: 334
    })
})


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import os

# # --- 1. Định nghĩa đường dẫn chính xác đến thư mục model và tokenizer trên Google Drive ---
# # Thư mục gốc của Google Drive là "/content/drive/My Drive/"
# # Sau đó là đường dẫn mà bạn đã tạo bên trong Drive
# model_directory = "/content/drive/My Drive/my_finetuned_model_on_drive/finetuned_model"
# tokenizer_directory = "/content/drive/My Drive/my_finetuned_model_on_drive/tokenizer"

# # --- 2. Kiểm tra xem các thư mục có tồn tại không (Rất quan trọng để tránh lỗi) ---
# if not os.path.exists(model_directory):
#     print(f"Lỗi: Thư mục model không tìm thấy tại {model_directory}")
#     print("Vui lòng kiểm tra lại đường dẫn trên Google Drive. Có thể tên thư mục bị sai chính tả hoặc chưa được tạo.")
#     exit() # Thoát chương trình nếu không tìm thấy thư mục

# if not os.path.exists(tokenizer_directory):
#     print(f"Lỗi: Thư mục tokenizer không tìm thấy tại {tokenizer_directory}")
#     print("Vui lòng kiểm tra lại đường dẫn trên Google Drive. Có thể tên thư mục bị sai chính tả hoặc chưa được tạo.")
#     exit() # Thoát chương trình nếu không tìm thấy thư mục

# print(f"Đang tải model từ Google Drive: {model_directory}")
# print(f"Đang tải tokenizer từ Google Drive: {tokenizer_directory}")

# # --- 3. Tải lại Tokenizer ---
# try:
#     tokenizer = AutoTokenizer.from_pretrained(tokenizer_directory)
#     print("Tokenizer đã được tải thành công!")

#     # --- 4. Tải lại Model ---
#     # `AutoModelForCausalLM` là loại model phù hợp cho các tác vụ sinh văn bản (causal language modeling),
#     # như Llama hoặc GPT-2. Nếu model của bạn là loại khác (ví dụ: phân loại văn bản),
#     # bạn cần dùng `AutoModelForSequenceClassification` hoặc tương tự.
#     model = AutoModelForCausalLM.from_pretrained(model_directory)
#     print("Model đã được tải thành công!")

#     print("\n--- Model đã sẵn sàng để sử dụng hoặc tiếp tục fine-tuning ---")

#     # --- 5. Ví dụ sử dụng model để sinh văn bản (để kiểm tra) ---
#     prompt = "Một ngày nắng đẹp, tôi quyết định đi "
#     print(f"Prompt đầu vào: '{prompt}'")

#     # Mã hóa đầu vào
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device) # Đảm bảo input nằm trên cùng thiết bị với model
#     # Nếu bạn đang dùng GPU (nên dùng), model.device sẽ là 'cuda:0'

#     # Sinh văn bản
#     # Có thể điều chỉnh các tham số sinh văn bản tùy theo nhu cầu
#     outputs = model.generate(
#         inputs.input_ids,
#         max_new_tokens=50, # Số lượng token mới tối đa được sinh ra
#         num_beams=1,       # Đặt >1 nếu muốn dùng beam search (kết quả có thể tốt hơn, nhưng chậm hơn)
#         do_sample=True,    # Có lấy mẫu ngẫu nhiên không (True = sáng tạo hơn, False = deterministic)
#         top_k=50,          # Chọn từ K token có xác suất cao nhất
#         top_p=0.95,        # Chọn từ tập token có xác suất tích lũy đến P%
#         temperature=0.7,   # Điều chỉnh độ ngẫu nhiên (sáng tạo), thấp hơn = ít ngẫu nhiên hơn
#         pad_token_id=tokenizer.pad_token_id, # Cần thiết cho một số model
#         eos_token_id=tokenizer.eos_token_id  # Dừng sinh khi gặp token kết thúc câu
#     )

#     # Giải mã kết quả
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("\nVăn bản được tạo ra:")
#     print(generated_text)

# except Exception as e:
#     print(f"\nĐã xảy ra lỗi trong quá trình tải model hoặc tokenizer: {e}")
#     print("Các nguyên nhân có thể:")
#     print(" - Chưa cài đặt các thư viện cần thiết (transformers, torch/tensorflow).")
#     print(" - Lỗi bộ nhớ (Out of Memory - OOM) nếu model quá lớn so với GPU/RAM.")
#     print(" - Phiên bản thư viện không tương thích.")
#     print(" - Có thể bạn cần cài đặt `accelerate` hoặc `bitsandbytes` nếu model lớn.")
#     # Cài đặt thêm nếu gặp lỗi OOM hoặc cần tối ưu
#     # !pip install accelerate bitsandbytes

Đang tải model từ Google Drive: /content/drive/My Drive/my_finetuned_model_on_drive/finetuned_model
Đang tải tokenizer từ Google Drive: /content/drive/My Drive/my_finetuned_model_on_drive/tokenizer
Tokenizer đã được tải thành công!

Đã xảy ra lỗi trong quá trình tải model hoặc tokenizer: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 3201 has 14.74 GiB memory in use. Of the allocated memory 14.33 GiB is allocated by PyTorch, and 276.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Các nguyên nhân có thể:
 - Chưa cài đặt các thư viện cần thiết (transformers, torch/tensorflow).
 - Lỗi bộ nhớ (Out of Memory - OOM) nếu model quá lớn so với GPU/RAM.
 - Phiên bản thư viện không tư

In [8]:
# Cấu hình tên model và nơi lưu
base_model_name = "bkai-foundation-models/vietnamese-llama2-7b-40GB"
finetune_model_name = "my_finetuned_vllama2"
#cấu hình vài tham số
output_dir = "./results"
#số lượng epoch
num_train_epochs = 3
#no change params
# Thông số cho quantization và LoRA
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

fp16 = False
bf16 = False

#Thông số huấn luyện
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 1.0

learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"

lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03

group_by_length = True
save_steps = 0
logging_steps = 25

max_seq_length = None
packing = False
neftune_noise_alpha = {"": 0}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Paste your Hugging Face token here: ")
login(token=hf_token)

Paste your Hugging Face token here: ··········


In [9]:
#cài đặt quantization dựa vào tham số ở trên
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
# Thiết lập quantization (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",             # nf4 là loại lượng tử tốt hơn int4
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False        # Không dùng nested quant để tiết kiệm RAM
)
# Load model với quantization
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",                     # Tự chia GPU/CPU nếu cần
    trust_remote_code=True,
    attn_implementation="eager"
)
# Load tokenizer tương ứng
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True
)

# Thiết lập pad_token để tránh lỗi nếu model không có sẵn
tokenizer.pad_token = tokenizer.eos_token

# Padding bên phải (giữ thống nhất định dạng đầu vào)
tokenizer.padding_side = "right"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
prompt = "<|user|>\n[Tác vụ: mô tả]\nBạn có thể kể tên một vài loại phân bón?.\n\n<|assistant|>\n"
# Dùng pipeline
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer)
# Sinh văn bản
result = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.8)
#temperature: độ sáng tạo
#max_new_tokens độ dài token
# In kết quả
print(result[0]['generated_text'])

Device set to use cuda:0


<|user|>
[Tác vụ: mô tả]
Bạn có biết tiếng Việt không?.

<|assistant|>
 Hinweis:


In [10]:
#cấu hình LoRA
#https://discuss.huggingface.co/t/task-type-parameter-of-loraconfig/52879
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

#set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,# vừa làm ở trên
    num_train_epochs=num_train_epochs,#1

    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,

    learning_rate=learning_rate,
    weight_decay=weight_decay,

    fp16=fp16,
    bf16=bf16,

    max_grad_norm=max_grad_norm,
    max_steps=max_steps,

    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none"  # Trong TrainingArguments
)

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=transformed_dataset["train"],
    peft_config=peft_config,
    args=training_arguments
)

trainer.train()

Adding EOS to train dataset:   0%|          | 0/3006 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3006 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3006 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
25,3.503600


In [ ]:
# Tạo pipeline
pipe = pipeline(
    "text-generation",
    model=base_model,                   # model đã merge
    tokenizer=tokenizer,
    #device=0                       # hoặc "cuda:0" nếu dùng GPU
    device_map="auto"
)

# Test thử
output = pipe("<|user|>\n[Tác vụ: trả lời câu hỏi]\nBạn có thể kể tên một vài loại phân bón?\n\n<|assistant|>\n", max_new_tokens=50, temperature=0.8, do_sample=False)
print(output[0]['generated_text'])

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|user|>
[Tác vụ: trả lời câu hỏi]
Bạn có thể kể tên một vài loại phân bón?

<|assistant|>
Có các loại phân bón như đạm, lân, kali, lân-hợp chất, phân vi sinh, phân bón hữu cơ, phân bón tổng hợp, phân bón sinh học, phân bón tổng hợp hữu cơ, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu cơ tổng hợp, phân bón hữu


In [ ]:
#save trained model:
trainer.model.save_pretrained(finetune_model_name)

In [ ]:
# # Tạo thư mục offload nếu chưa có
# import os
# os.makedirs("./offload", exist_ok=True)

# # Config cho mô hình 4bit
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=False
# )
#merge model gốc và LoRA
for var in ["base_model", "pipeline", "trainer", "model"]:
    if var in globals():
        del globals()[var]
#reload and merge
device_map = "auto"
# Load lại base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    quantization_config=bnb_config,
    device_map=device_map,
    offload_folder="./offload"  # <== thêm dòng này
)

# Nạp LoRA đã fine-tuned vào base model
model = PeftModel.from_pretrained(
    base_model,
    finetune_model_name,
    device_map="auto",
    offload_folder="./offload"
)
model = model.merge_and_unload()  # Merge adapter vào base

# Reload tokenizer để lưu lại
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
#save lại model
# Lưu mô hình đã fine-tune
model.save_pretrained("models/finetuned_model")

# Lưu tokenizer tương ứng
tokenizer.save_pretrained("models/tokenizer")


('models/tokenizer/tokenizer_config.json',
 'models/tokenizer/special_tokens_map.json',
 'models/tokenizer/tokenizer.json')

In [ ]:
!zip -r /content/models.zip models/

  adding: models/ (stored 0%)
  adding: models/finetuned_model/ (stored 0%)
  adding: models/finetuned_model/generation_config.json (deflated 35%)
  adding: models/finetuned_model/config.json (deflated 56%)
  adding: models/finetuned_model/model.safetensorsfrom google.colab import files files.download('/content/models.zip')
 (deflated 10%)
  adding: models/tokenizer/ (stored 0%)
  adding: models/tokenizer/special_tokens_map.json (deflated 72%)
  adding: models/tokenizer/tokenizer_config.json (deflated 68%)
  adding: models/tokenizer/tokenizer.json (deflated 85%)


In [ ]:
from google.colab import files
files.download('/content/models.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Giả sử bạn đã chạy lệnh save_pretrained trước đó
# model.save_pretrained("models/finetuned_model")
# tokenizer.save_pretrained("models/tokenizer")

# Bây giờ, di chuyển hoặc copy nó lên Google Drive của bạn
# Tạo một thư mục trên Drive để chứa model của bạn
!mkdir -p "/content/drive/My Drive/my_finetuned_model_on_drive/finetuned_model"
!mkdir -p "/content/drive/My Drive/my_finetuned_model_on_drive/tokenizer"

# Copy các file từ Colab VM sang Google Drive
!cp -r models/finetuned_model/* "/content/drive/My Drive/my_finetuned_model_on_drive/finetuned_model/"
!cp -r models/tokenizer/* "/content/drive/My Drive/my_finetuned_model_on_drive/tokenizer/"

print("Model và tokenizer đã được lưu lên Google Drive.")

Model và tokenizer đã được lưu lên Google Drive.
